In [14]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [15]:
data = pd.read_csv('Data\emotion.csv')

In [16]:
data

,text,emotion
0,carefully word blog posts amount criticism hea...,0
1,cannot remember little mermaid feeling carefre...,1
2,not feeling super well turns cold knocked next...,1
3,feel honored part group amazing talents,1
4,think helping also began feel pretty lonely lo...,0
...,...,...
254534,feeling generally jaded nothing like full blow...,0
254535,know feel shitty crappy regretful disorganized...,0
254536,miss boo another note soready game come tonigh...,0
254537,started feel real handsome,1


In [41]:
text_test = data['text'][10]

text_test

'feeling sensation divine love chest'

In [43]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(text_test)
    
endpoint = "http://localhost:8080/v1/models/emotion-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Happy" if prediction_value > 0.6 else "Not Happy"
else:
    result = "Error: No predictions found in response."

print(result)

Happy


In [47]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/emotion-prediction-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1704120234'}]}
